# 16.413 Final Project: Convex Risk Bounded Continuous-Time Trajectory Planning for a Europa Lander

Julia Briden, Kota Kondo, Lakshay Sharma, Nick Rober, and Youngjae Min

In [ ]:
# ## 1. Problem Formulation

# Github code from paper: https://github.com/jasour/Risk-Bounded-Continuous-Time-Trajectory-Planning/blob/main/RRT_Tools/RRT_SOS_Static.m


# Determine obstacles (dynamic or static, convex or nonconvex), start and goal points, probabilistic optimization problem formulation, constraints, planning time horizon

# https://trs.jpl.nasa.gov/bitstream/handle/2014/47541/CL%2317-0518.pdf?sequence=1

# Precision Landing Maneuver:
# Upon receiving an initial surface-relative position and velocity estimate from the ILS, guidance
# will perform the Precision Landing Maneuver (PLM). The objective of this maneuver is to steer
# the PDV so that, at the end of the maneuver, the PDV is vertically-oriented directly above the
# landing site at an altitude of 1000 m with zero horizontal velocity and vertical velocity of -30 m/s.
# When the PLM is complete, the PDV will descend to an altitude of 500 m to begin Hazard
# Detection and Avoidance.

# Hazard Detection and Avoidance:
# When the PDV is at an altitude of 500 m and directly above the targeted landing location, it uses
# the 3D-imaging lidar to scan the landing site, construct a safe landing map, and select a landing 
# Pre-decisional: for information and discussion only 13
# location within that map. This activity must complete in three seconds, at which time the PDV will
# be approximately 400 m above the targeted landing location.
# Once the Hazard Detection activity has finished, guidance uses the new, safe landing location
# to construct a guidance profile and steer the vehicle to a point above the targeted safe landing spot.
# The Hazard Avoidance Maneuver begins at an altitude of 250 m and completes at an altitude of
# 30 m with a vertical velocity of -0.5 m/s. The safe landing map, using the data from the 3D-imaging
# lidar, will be 100 m  100 m and the vehicle has the capability to land anywhere within that map.

# ![alt text](europalander.png "Europa Lander mission concept")

## 1. Problem Statement

The year is 2048 and alien life has been detected under the ice of Europa! In the interest of getting to know our newly discovered neighbors, NASA has decided to dispatch a highly specialized fleet of vehicles to the Gallilean moon. The mission will consist of (i) an autonomous underwater vehicle (AUV) designed to slowly glide to Europa's sea floor while transmitting data via an optical fiber cable to a rover on the surface, (ii) a surface rover designed to ferry the underwater vehicle from a safe landing site to an appropriate insertion site (i.e. vapor plume outburst location), and (iii) an orbiter, capable of relaying signals from the rover and providing high-resolution imagery to map the surface.

![alt text](environment_basic.jpg "Europa Lander mission concept")

As part of NASA's Jet Propulsion Laboratory, our team has been tasked with creating the trajectory generation algorithm for the surface vehicle. The goal for the rover is to navigate from a safe landing site which is flat and free of obstacles to the AUV's insertion point, which has relatively thin ice and may be surrounded by uncertain hazards. We have access to global map data within the field of operation, allowing us to formulate the trajectory generation problem as an infinite horizon mixed-integer nonlinear programming problem. We will solve this optimization problem using scipy.minimize and the Bernstein polynomial library BEBOT. To provide additional autonomy capabilities and give the optimization algorithm a good chance of finding a feasible solution, we will initilize the optimizer using RRT* to form an initial guess of the optimal trajectory.

### Formulation

Generally, this problem can be stated as an optimal control problem in the form of

***
<font size="3"><h1><center>$\underset{x(t)\in\mathbb{R}^{n_x},\;u(t)\in\mathbb{R}^{u_n}}{\text{minimize}}\quad E(x(t_0),\;x(t_f))+\int_{t_0}^{t_f}F(x(t),u(t))dt$ <div style="text-align: right"> (1) </div><br> $\begin{align} & \text{subject to} \quad \quad \\  \\ &\dot{x} = f(x(t),\;u(t)),\quad \forall t\in[t_0,t_f] \\ & e(x(t_0),\;x(t_f)) = 0, \\ & h(x(t),\;u(t)) \leq 0,\quad \forall t\in[t_0,t_f] \end{align}$</center></h1></font>
***

However, this is a difficult problem to solve. We can simplify it by considering that our system can be modelled using the common unicycle model:

***
<font size="3"><h1><center>$\dot{x} = \begin{bmatrix} \dot{x}_1 \\ \dot{x}_2 \\ \dot{\phi} \end{bmatrix} = \begin{bmatrix} V \cos(\phi) \\ V \sin(\phi) \\ \omega \end{bmatrix}$</center></h1></font>
***

which is differentially flat. In this case, that is helpful because it means we can determine the inputs $V$ and $\omega$ directly from $x_1$ and $x_2$. Additionally, we can approximate $x(t)$ (the optimal trajectory) using a  Bernstein polynomial, which takes the form 

***
<font size="3"><h1><center>$x_N(t) = \sum_{i=0}^N \bar{x}_{i,N}b_{i,N}(t)$</center></h1></font>
***

where $\bar{x}_N = [\bar{x}_{0,N},\; ...,\; \bar{x}_{N,N}] \in \mathbb{R}^{n_x \times (N+1)}$, is a vector of control points for the Bernstein polynomial and 

***
<font size="3"><h1><center>$b_{i,N}(t) = \begin{pmatrix}N \\ i\end{pmatrix}t^i(1-t)^{N-i},\quad \begin{pmatrix}N \\ i\end{pmatrix} = \frac{N!}{i!(N-i)!}$</center></h1></font>
***

The use of Bernstein polynomials allows us to formulate the problem as a finite-dimensional optimization problem (choose N control points), rather than an infinite dimensional optimization problem. Additionally, Bernstein polynomials are fixed at the endpoints, (i.e. $x_N(t_0) = \bar{x}_{0,N}$ and $x_N(t_f) = \bar{x}_{N,N}$, allowing us to remove the equality constraint from (1). The fact that the model is differentially flat allows us to remove $u(t)$, and the convex hull property of Bernstein polynomials allows us to only evaluate the control points $\{x_{0,N}, ..., x_{N,N}\}$ for satisfaction of the inequality constraints. Thus, the simplified problem becomes

***
<font size="3"><h1><center>$\underset{\bar{x}_N}{\text{minimize}}\quad E(\bar{x}_{0,N},\;\bar{x}_{N,N})+\int_{t_0}^{t_f}F(x_N(t))dt$ <div style="text-align: right"> (2) </div><br> $\begin{align} & \text{subject to} \quad & \quad \\ & \|\dot{x}_N(t_i) - f(x_N(t_i)\| \leq 0, & \forall i \in \{0, 1, ..., N\} \\ & h(\bar{x}_{i,N}) \leq 0, & \forall i \in \{0, 1, ..., N\} \end{align}$</center></h1></font>
***

Finally, to address the inequality constraints $h(\bar{x}_{i,N})$, we list the necessary constraints on the system as

- Collision avoidance constraints (addressed below)
- Maximum and minimum speed constraints
- Maximum and minimum angular rate constraints

and plan to use total path length as the cost function to minimize the distance required by the rover to travel to the AUV insertion point.


From a practical standpoint, the scipy function minimize() can be used to solve problem (2). However, it requires an initial guess of the optimal trajectory as a starting point from which to begin using gradient descent. This guess warrants some consideration as it can have a large influence on the output of the function: one initial guess may give a good answer, while another may give local minima or even fail to converge to a solution. One option to generate this initial guess is to linearly interpolate between the initial and final positions. However it is often beneficial to give an initial guess which satisfies the system's constraints, which may not be the case if an obstacle lies on the straightline path between the initial and final positions. Thus, we plan to use RRT* as a fast way to generate what is hopefully a good initial guess.





## 2. Risk Contours

Risk contours are defined as the set of all points in uncertain environments with guaranteed bounded risk. We will use risk contours in finding optimal trajectories. 

For simplicity we incorporate only static risk contours into our problem, not dynamic contours, and replace probabilistic constraint with deterministic constraint in terms of x (use inner approximation of probabilisitic constraint to bound risk), which lead to rational polynomial representation.

### How to construct risk contours

Define $\chi \in \mathbb{R}^2$ as an uncertain environment where our ground robot explores and $\chi_{obs_{i}}(\omega_i) \subset \chi$, for $i = 1, ..., n_{o_{s}}$ as static uncertain obstacles, where $n_{o_{s}}$ is the number of static obstacles, $\omega_i$ is a probablistic uncertain parameters. 

Then, we can represent $\chi_{obs_{i}}(\omega_i)$ in terms of polynomials in x $\in \chi$

***
$\chi_{obs_{i}}(\omega_i) \equiv \{x \in \chi | P_i(x, \omega_i) \geq 0\}$ for i = 1, ..., $n_{o_{s}}$ (1)
***

where P_i : $\mathbb{R}^{n_x + n_{\omega}} \to \mathbb{R}$ denotes a given polynomial. Let $\triangle \in [0, 1]$ be a given acceptable risk level, and then, $C^{\triangle}_{r}$, static risk contour whose risk level, with less than the given acceptable can be defined as

***
$C^{\triangle}_{r} \equiv \{x \in \chi | Prob(x \in \chi_{obs}(\omega)) \leq \triangle\}$ (2)
***

By this definition, static risk contour is now defined with a deterministic constraint in terms of x. Furthermore, the static risk contour's inner approximation is given as

***
$\hat{C}^{\triangle}_{r} \equiv \{x  \in \chi | \frac{\mathbb{E}[P^2(x, \omega)] - \mathbb{E}[P(x, \omega)]^2}{\mathbb{E}[P^2(x, \omega)]} \leq \triangle  or  \mathbb{E}[P(x, \omega)] \leq 0 \}$ (3)
***

Since $\hat{C}^{\triangle}_{r}$ is an approximation of inner space of $C^{\triangle}_{r}$, any trajectory going through $\hat{C}^{\triangle}_{r}$ has a risk less or equal to the given $\triangle$.

*Convex Risk Bounded Continuous-Time Trajectory Planning in Uncertain Nonconvex Environments (https://arxiv.org/pdf/2106.05489.pdf)* gives a good expample to illustrate the use of this risk contour generation method.

### Function outline for generating random obstacles in the environment

In [25]:
## Function definition
#
# input
#    env: environment size [x1, y1, x2, y2]
#    shape: obstacle shape options (eg. circle, rectangle, etc)
#.   max_size: maximum obstacle size [x_max, y_max]
#    omega_range: range of potential probablistic uncertain parameters [min, max]
#
# output
#    pols: a list of polynomials that represents an obstacle, including probabilistic uncertain parameter, omega
#

def random_ob(env, shape, max_size, omega_range):
    # randomly generate obstacles within the given environments
    # return a list of polynomials generated according to the given shape, maximum size, and omega range
    return pols

### Function outline for static risk contours 

In [26]:
## Function definition
#
# input
#    delta: acceptable risk level, [0,1]
#    pol: polynomial that represents an obstacle, including probabilistic uncertain parameter, omega  
#
# output
#    C: the boundary of inner approximation of risk contour, which is denoted as a polynomial
#

def static_rc(delta, pol):
    # calculate C using equation (3)
    # need to calculate expected values using the given polynomials
    # since delta is given, we can find the boundary of C as a form of a polynomial
    # may need another function to convert the given polynomial to another form. 
    return C

## 3. Continuous-Time Risk Bounded Trajectory Planning using Risk Contours


From part 2, we have formulated conditions for a point $x$ in a trajectory to be safe with the maximum risk of $\Delta$ to collide with uncertain static obstacles as below.

***
$\frac{\mathbb{E}[P^2(x, \omega)] - \mathbb{E}[P(x, \omega)]^2}{\mathbb{E}[P^2(x, \omega)]} \leq \Delta  \land  \mathbb{E}[P(x, \omega)] \leq 0 \$
***

With known momentums of $\omega$, the conditions are polynomial in $x$, and then the feasible set for the trajectory optimization problem can be presented as

***
$\mathcal{S}=\{x: g_i(x) \geq 0 \;\; i=1,\dots,l\}$
***

for some polynomials $g_i$. We are using RRT to find a trajectory in this feasible set to reach the goal state from the initial state. For RRT, we have to define two important functions STEER and ObstacleFree.

### Steer

Given a sampled point and the current state of the RRT tree, steer() finds the node in the RRT tree nearest to the sampled point, and generates a path from that node of length up to $d$ in a straight line towards the sampled point. This path is added as an edge and the endpoint of this path is added as a new node to the RRT tree. If the distance to the sampled point from the nearest node is less than $d$, the path terminates at the sampled point, and if the path encounters an obstacle during generation, the RRT tree is returned unchanged.



In [ ]:
## Function definition
#
# input
#    tree: current RRT tree
#    endpoint: sampled point towards which the RRT tree needs to grow
#    d: maximum path length of the new edge
#    env: environment representation, including obstacles 
#
# output
#    None (steer modifies the input tree in place)
#

def steer(tree, endpoint):
    return 

### ObstacleFree

To check the safety of the path defined above, one can use SOS-based continuous-time technique by checking whether each polynomial $g_i(x(t))$ can be represented with SOS polynomials as

***
$g_i(x(t)) = \sigma_{0i}(t) + \sigma_{1i}(t)(t-t_1) + \sigma_{2i}(t)(t_2-t)$
***

where $\sigma_{0i}(t), \sigma_{1i}(t), \sigma_{2i}(t)$ are SOS polynomials. This can be checked using Yalmip and Spotless packages, but for simplicity, we employ an apporximate collision-checking method.

We sample points along the extended path, and check for each point whether it resides in $\mathcal{S}$. By sampling densely enough, we can approximately check if the exented path is free from collision with obstacles.

In [ ]:
def ObstacleFree():
    return

### RRT for trajectory generation

Then, using the functions defined above, we can employ RRT to generate a trajectory with bounded risk $\Delta$ to collide with uncertain static obstacles.

In [ ]:
def generate_trajectory():
    return